# Parte 3 - Machine Learning Workflow

Datasets: [Diamanti](https://www.kaggle.com/shivam2503/diamonds)

**OBBIETTVO:** In base alle sue caratteristiche provare a predire il prezzo di un diamante <br>
Utilizzeremo la libreria python **scikit-learn** per testare alcuni algoritmi di classificiazione!

In [1]:
import pandas as pd
import numpy as np

import sklearn
from sklearn import svm, preprocessing

diamond_df = pd.read_csv("../datasets/diamonds.csv", index_col = 0)

In [2]:
diamond_df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


**Quale modello algoritmo di classificazione dovremmo utilizzare?**

[Come scegliere il corretto algoritmo](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)

In [3]:
len(diamond_df)

53940

## Linear Regression

<img src="../images/regression_1.jpeg" alt="Drawing" style="width: 545px;"/><img src="../images/regression_2.jpeg" alt="Drawing" style="width: 500px;"/> <br> Per utilizzare la Linear Regression deve esserci una relazione lineare tra i dati

<img src="../images/regression_3.png" alt="Drawing" style="width: 545px;"/> 

[Least Square Method](https://www.varsitytutors.com/hotmath/hotmath_help/topics/line-of-best-fit)

Come si definisce la best fit line? <br>

$$Y = mx + b$$

Data una X dobbiamo trovare la sua Y corrispondente, ma prima dobbiamo risolvere _m_ e _b_:

_m_ è la pendenza<br>
_b_ è l'intersezione di y

$$m = \frac{\overline{x}\cdot\overline{y} - \overline{xy}}{(\overline{x})^2-\overline{x^2}}$$

$$b = \overline{y}-m\overline{x} $$

In questo caso abbiamo dati su 2 dimensioni, ma appena incrementiamo la dimensione dello spazio verriale incrementerà anche la complessità dei calcoli

**Per allenare il nostro modello vogliamo utilizzare tutti i paramentri tranne il prezzo**

In [4]:
diamond_df["cut"].unique()

array(['Ideal', 'Premium', 'Good', 'Very Good', 'Fair'], dtype=object)

Abbiamo bisogno di categorie numeriche!

In [5]:
diamond_df["cut"].astype("category").cat.codes

1        2
2        3
3        1
4        3
5        1
        ..
53936    2
53937    1
53938    4
53939    3
53940    2
Length: 53940, dtype: int8

**Problema**: Dobbiamo preservare il significato delle labels: per esempio Premium sarà migliore di Fair e così via..

In [6]:
cut_dizionario = {"Fair":1, "Good":2, "Very Good":3, "Premium":4, "Ideal":5}

Stessa cosa per:

In [7]:
clarity_dizionario = {"I3": 1, "I2": 2, "I1": 3, "SI2": 4, "SI1": 5, "VS2": 6, "VS1": 7, "VVS2": 8, "VVS1": 9, "IF": 10, "FL": 11}
color_dizionario = {"J": 1,"I": 2,"H": 3,"G": 4,"F": 5,"E": 6,"D": 7}

Bisognerà mappare queste classi alle varie colonne del dataset

In [8]:
diamond_df['cut'] = diamond_df['cut'].map(cut_dizionario)

## Esercizio 13:
- Mappare le colonne "clarity" e "color" con i rispettivi dizionari!

In [9]:
#Esercizio
diamond_df['clarity'] = diamond_df['clarity'].map(clarity_dizionario)
diamond_df['color'] = diamond_df['color'].map(color_dizionario)

In [10]:
diamond_df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
1,0.23,5,6,4,61.5,55.0,326,3.95,3.98,2.43
2,0.21,4,6,5,59.8,61.0,326,3.89,3.84,2.31
3,0.23,2,6,7,56.9,65.0,327,4.05,4.07,2.31
4,0.29,4,2,6,62.4,58.0,334,4.20,4.23,2.63
5,0.31,2,1,4,63.3,58.0,335,4.34,4.35,2.75


Prima di allenare il nostro modello è importante mescolare i dati per evitare che si formi del biasing analizzando i dati in ordine.
<br> Per esempio potrebbe essere, come nel nostro caso, che i dati siano ordinati per prezzo.

In [11]:
diamond_df

,carat,cut,color,clarity,depth,table,price,x,y,z
1,0.23,5,6,4,61.5,55.0,326,3.95,3.98,2.43
2,0.21,4,6,5,59.8,61.0,326,3.89,3.84,2.31
3,0.23,2,6,7,56.9,65.0,327,4.05,4.07,2.31
4,0.29,4,2,6,62.4,58.0,334,4.20,4.23,2.63
5,0.31,2,1,4,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53936,0.72,5,7,5,60.8,57.0,2757,5.75,5.76,3.50
53937,0.72,2,7,5,63.1,55.0,2757,5.69,5.75,3.61
53938,0.70,3,7,5,62.8,60.0,2757,5.66,5.68,3.56
53939,0.86,4,3,4,61.0,58.0,2757,6.15,6.12,3.74


In [12]:
diamond_df = sklearn.utils.shuffle(diamond_df)

Separiamo il set di features dalla label che dobbiamo predire

In [13]:
X = diamond_df.drop("price", axis=1).values #Feature Set --> Ogni label tranne quella che dobbiamo predire
y = diamond_df["price"].values

In [14]:
X

array([[0.36, 5.  , 5.  , ..., 4.57, 4.6 , 2.82],
       [0.72, 5.  , 3.  , ..., 5.72, 5.76, 3.57],
       [1.53, 3.  , 6.  , ..., 7.43, 7.51, 4.54],
       ...,
       [0.32, 5.  , 3.  , ..., 4.46, 4.42, 2.68],
       [1.7 , 5.  , 2.  , ..., 7.57, 7.5 , 4.74],
       [0.4 , 2.  , 5.  , ..., 4.69, 4.72, 2.91]])

**Bonus**: Avremmo potuto barare caricando il dataframe con l'index, poiché il dataset era ordinato per prezzo e facendo ciò avremmo lasciato un informazione in più che avrebbe compromesso la nostra regressione, poiché l'indice sarebbe stato ordinato come il prezzo.

**Preprocessing**: Permette di normalizzare i valori, in questo modo ridurremo la sparsità e il modello lavorerà con dati più uniformi con conseguente miglioramente nelle performance

In [15]:
print(np.mean(X))

16.59665167058048


In [16]:
X = preprocessing.scale(X) #Cerca

In [17]:
print(np.mean(X))

2.073404275280678e-16


In [18]:
test_size = 200

Il **train** è una porzione del dataset per cui il modello viene allenato

Il **test** è una porzione del dataset che il nostro modello non vedrà mai, e sarà usato per valutarne le performance.

In [19]:
X_train = X[:-test_size]
y_train = y[:-test_size]

X_test = X[-test_size:]
y_test = y[-test_size:]

Nel caso utilizzassimo il dataset di test per allenare il modello l'accurattezza finale risulterebbe compromessa poiché i dati sono già stati visionati dal modello durante la fase di train, in questo modo non possiamo verificare tramite il test se il modello abbia realmente imparato a predire un valore o abbia solamente imparato a memoria il dataset di train. Per questo motivo i dati di test devono essere utilizzati solo per testare il modello!

Andiamo a selezionare il [modello](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) da utilizzare

In [20]:
clf = svm.SVR(kernel="linear")
clf.fit(X_train, y_train)

SVR(kernel='linear')

In [21]:
clf.score(X_test, y_test)

0.8414791342579695

Coefficiente di determinazione (R quadro): 
- 0.0 Caso peggiore 
- 1.0 Caso migliore

Esso è calcolato partendo dall'errore quadratico medio

Fin troppo bello per essere vero!

Andiamo a verificare quello che è successo!

In [22]:
for X,y in zip(X_test, y_test):
    print(f"Model: {clf.predict([X])}, Actual: {y}")
    

Model: [2759.99918279], Actual: 2301
Model: [68.6001562], Actual: 502
Model: [558.61638693], Actual: 1000
Model: [2251.0386317], Actual: 1928
Model: [1.85988253], Actual: 605
Model: [3222.6810463], Actual: 2681
Model: [8719.06071473], Actual: 9557
Model: [2850.84278289], Actual: 2057
Model: [4507.0237187], Actual: 4360
Model: [3369.20090063], Actual: 2763
Model: [1433.35420337], Actual: 974
Model: [912.69871768], Actual: 752
Model: [1196.32607346], Actual: 844
Model: [138.08955146], Actual: 748
Model: [6548.81230762], Actual: 6552
Model: [1758.43962677], Actual: 1356
Model: [6498.59698363], Actual: 4278
Model: [9996.72536659], Actual: 11737
Model: [5277.61065347], Actual: 5706
Model: [190.47019139], Actual: 776
Model: [-66.14398432], Actual: 550
Model: [4352.48547196], Actual: 4381
Model: [3786.46435075], Actual: 3551
Model: [5470.73125706], Actual: 6830
Model: [5610.6528237], Actual: 5008
Model: [170.2406006], Actual: 590
Model: [77.90959764], Actual: 561
Model: [-168.83362517], Actua

Testiamo un altro modello!

In [23]:
clf = svm.SVR(kernel="rbf")
clf.fit(X_train, y_train)

SVR()

In [24]:
clf.score(X_test, y_test)

0.6250887260458173

In [25]:
for X,y in zip(X_test, y_test):
    print(f"Model: {clf.predict([X])}, Actual: {y}")

Model: [3309.28030506], Actual: 2301
Model: [833.2631206], Actual: 502
Model: [1384.52801449], Actual: 1000
Model: [1995.82907062], Actual: 1928
Model: [468.36576844], Actual: 605
Model: [2953.58804944], Actual: 2681
Model: [5778.00215426], Actual: 9557
Model: [2541.0010444], Actual: 2057
Model: [4419.6195572], Actual: 4360
Model: [3230.41498623], Actual: 2763
Model: [1176.39301644], Actual: 974
Model: [1022.73555572], Actual: 752
Model: [1485.0797959], Actual: 844
Model: [1521.27167801], Actual: 748
Model: [5969.38011599], Actual: 6552
Model: [1364.71210087], Actual: 1356
Model: [5935.85669044], Actual: 4278
Model: [6209.74015744], Actual: 11737
Model: [4054.02797279], Actual: 5706
Model: [670.48833993], Actual: 776
Model: [731.54192132], Actual: 550
Model: [4033.81008915], Actual: 4381
Model: [3607.4432458], Actual: 3551
Model: [4814.81467324], Actual: 6830
Model: [5547.83819048], Actual: 5008
Model: [1235.8256547], Actual: 590
Model: [268.69025255], Actual: 561
Model: [647.5085336],

**BONUS:** Si potrebbero usare più classifier e alla fine fare una media delle prestazioni!